In [ ]:
import sys
sys.path.append('..')
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout, Dropdown
from IPython.display import display, clear_output, HTML
import time
import traceback
from src.core import (
    process_order_request, 
    process_ride_plan_request, 
    process_event_plan_request
)

# --- Define User Queries for All Domains ---
tax_queries = [
    "We had two burgers at 10 euros each and a large soda for 3 euros."
]
cycling_queries = [
    "I want to plan a long, flat ride. Let's say 100 kilometers."
]
event_queries = [
    "Let's plan our AI Summit. We need to book Dr. Sharma for a session called 'Intro to Transformers'. It will be in the Grand Ballroom, which has a capacity of 500 and A/V. We expect 450 people.",
    "Let's plan our AI Summit. We need a 'Deep Dive into NLP' workshop in Workshop Room A, which has a capacity of 40 but no A/V system. The workshop requires A/V and will have 35 attendees."
]
print("✅ System ready. Three domains are configured.")

In [ ]:
def create_multi_domain_ui():
    domain_selector = Dropdown(
        options=['Tax Calculation', 'Cycling Ride Planner', 'Event Planning'],
        value='Tax Calculation',
        description='Select Domain:',
        style={'description_width': 'initial'}
    )
    query_input = Textarea(
        value=tax_queries[0],
        description='User Query:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )
    run_button = Button(description='Generate DSL & Execute', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    def on_domain_change(change):
        domain = change['new']
        if domain == 'Tax Calculation': query_input.value = tax_queries[0]
        elif domain == 'Cycling Ride Planner': query_input.value = cycling_queries[0]
        elif domain == 'Event Planning': query_input.value = event_queries[0]
    domain_selector.observe(on_domain_change, names='value')

    def run_analysis(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'LLM processing...'; progress_bar.layout.visibility = 'visible'; progress_bar.value = 5

            domain = domain_selector.value
            query = query_input.value
            
            result = {}
            if domain == 'Tax Calculation': result = process_order_request(query)
            elif domain == 'Cycling Ride Planner': result = process_ride_plan_request(query)
            elif domain == 'Event Planning': result = process_event_plan_request(query)

            progress_bar.value = 10
            
            if result.get('llm_generated_dsl'):
                display(HTML("<h4>1. LLM-Generated DSL Code</h4>"))
                display(HTML(f"<pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{result['llm_generated_dsl']}</pre>"))

            if result["status"] == "success":
                display(HTML("<h4>2. Interpreter Execution Result</h4>"))
                dsl_result = result['interpreter_result']
                # Handle different domain results
                if domain == 'Event Planning':
                    sessions_html = ''.join([f"<li><b>{s['name']}</b> in {s.get('in_venue', 'N/A')} for {s.get('expected_attendees', 0)} people.</li>" for s in dsl_result['sessions']])
                    display(HTML(f"""<b>✅ Plan '{dsl_result['plan_name']}' Validated Successfully!</b>
                        <ul>{sessions_html}</ul>"""))
                else:
                     display(HTML(f"<p style='color:green; font-weight:bold;'>Validation Successful!</p><pre>{dsl_result}</pre>"))
            else:
                display(HTML(f"<h4 style='color:red;'>🔴 PROCESSING ERROR</h4><p>{result['message']}</p>"))

            run_button.disabled = False; progress_bar.layout.visibility = 'hidden'; progress_bar.value = 0
            
    run_button.on_click(run_analysis)
    print("--- Multi-Domain DSL Validator ---")
    # Initialize the UI with the first domain's query
    on_domain_change({'new': domain_selector.value})
    display(VBox([domain_selector, query_input, run_button, progress_bar, output_area]))

create_multi_domain_ui()